In [ ]:
;pwd

In [ ]:
]activate ..

In [ ]:
using Revise

In [ ]:
using Apophis

In [ ]:
# using TaylorIntegration, LinearAlgebra # Apophis.jl @reexports TaylorIntegration and LinearAlgebra
using Plots, DelimitedFiles, Dates
using Statistics: mean, std
using AstroTime

In [ ]:
using JLD
#recover integration from .jld file
vars = ["tv_jpl_integ", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
filename = string("Apophis_jt.jld")
for i in eachindex(vars)
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])
end

In [ ]:
apophis_radar_data_2005_2013 = process_radar_data_jpl("../Apophis_JPL_data.dat");

In [ ]:
apophis_data = apophis_radar_data_2005_2013[6:end] #take only the 2012-2013 subset

In [ ]:
#monostatic mode: check that each receiver and transmitter are the same
@show all( ismonostatic.(apophis_data) )

In [ ]:
#then, get the station codes
station_codes = [x.rcvr for x in apophis_data];

In [ ]:
#extract transmitter frequencies (MHz)
transmitter_freq_MHz = [x.freq for x in apophis_data];

In [ ]:
# get UTC receive times
tv_jpl_utc_jul = [datetime2julian(x.utcepoch) for x in apophis_data];

[x.utcepoch for x in apophis_data]

Let $\vec R$ denote the geocentric position of the observing station, $\vec \rho$ the topocentric position of the asteroid relative to the same station,  $\vec r_\mathrm{a}$ the geocentric position of the asteroid and $\vec r_\mathrm{E}$ the geocentric position of the asteroid. Then we have

$$
\vec r_\mathrm{a} =\vec r_\mathrm{E} + \vec R + \vec \rho
$$

Thus, the instantaneous topocentric range $\rho = |\vec \rho|$ of the asteroid may be computed as

$$
\rho = |\vec r_\mathrm{a} - \vec r_\mathrm{E} - \vec R| = \sqrt{(x_\mathrm{a} - x_\mathrm{E} - X)^2+(y_\mathrm{a} - y_\mathrm{E} - Y)^2+(z_\mathrm{a} - z_\mathrm{E} - Z)^2}
$$

And the instantaneous range rate $\dot \rho$ may be computed as:

$$
\dot \rho = \frac{1}{\rho}\vec{\rho} \cdot \dot{\vec{\rho}}
$$

#### Questions
- Is TDB the actual independent variable in the integration of the Solar System's equations of motion? ANSWER: YES
- Where can I get the integrated values (evolution?) of the orientation of the Moon, so that I do not have to integrate that from the beginning (1969)?

# Delay/Doppler observations reduction

In [ ]:
niters = 3 # number of iterations in time delay recursive process
rad_ind = 2

In [ ]:
station_codes[rad_ind]

In [ ]:
UTCEpoch.(TDBEpoch.(tv_jpl_integ, origin=:julian))

In [ ]:
apophis_data[rad_ind].utcepoch

In [ ]:
tdelay, dshift = delay_doppler(
    xv1[rad_ind+1, :],
    station_codes[rad_ind],
    apophis_data[rad_ind].utcepoch,
    transmitter_freq_MHz[rad_ind],
    niters
)

In [ ]:
tdelay_jpl, dshift_jpl = delay_doppler_jpleph(
    station_codes[rad_ind],
    apophis_data[rad_ind].utcepoch,
    transmitter_freq_MHz[rad_ind],
    niters
)

In [ ]:
apophis_data[rad_ind].delay # observed value

In [ ]:
1.0268298605e8 #μseconds
0.0000003025 * 1.0268298605e8 # magicfactor*delay μseconds

In [ ]:
tdelay_jpl #computed value

In [ ]:
tdelay()

In [ ]:
apophis_data[rad_ind].delay - tdelay() #O-C, TaylorIntegration

In [ ]:
apophis_data[rad_ind].delay - tdelay_jpl #O-C, JPL

In [ ]:
dshift_jpl

In [ ]:
dshift()

In [ ]:
apophis_data[rad_ind].doppler

In [ ]:
apophis_data[rad_ind].doppler - dshift() #TaylorIntegration

In [ ]:
apophis_data[rad_ind].doppler - dshift_jpl #O-C, JPL

In [ ]:
size(xv1)

In [ ]:
size(apophis_data)

In [ ]:
#get time-delay, Doppler-shift values from integration

tdelay_v1 = Array{Taylor1{Float64}}(undef, length(apophis_data))
dshift_v1 = Array{Taylor1{Float64}}(undef, length(apophis_data))

for i in eachindex(apophis_data)
    tdelay_v1[i], dshift_v1[i] = delay_doppler(
        xv1[i+1, :],
        apophis_data[i].rcvr,
        apophis_data[i].utcepoch,
        apophis_data[i].freq,
        niters
    )
end


In [ ]:
# compute time-delay, Doppler-shift values from JPL ephemerides

tdelay_JPL_v1 = Array{Float64}(undef, length(apophis_data))
dshift_JPL_v1 = Array{Float64}(undef, length(apophis_data))

for i in eachindex(apophis_data)
    tdelay_JPL_v1[i], dshift_JPL_v1[i] = delay_doppler_jpleph(
        apophis_data[i].rcvr,
        apophis_data[i].utcepoch,
        apophis_data[i].freq,
        niters
    )
end

# Time-delay / Doppler shift plots

In [ ]:
delay_index = findall(x->x.delay_units=="us", apophis_data)
doppler_index = findall(x->x.doppler_units=="Hz", apophis_data);

In [ ]:
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

In [ ]:
# Observed minus computed (O-C) residuals: JPL DE430+s199

# absolute
residual_JPL_td = tdelay_jpl_obs - tdelay_JPL_v1[delay_index]  # (usec) <- 1.000000302 magic factor
residual_JPL_ds = dshift_jpl_obs - dshift_JPL_v1[doppler_index] # (Hz)

# relative
rel_res_JPL_td = residual_JPL_td ./ tdelay_jpl_obs # (usec)
rel_res_JPL_ds = residual_JPL_ds ./ dshift_jpl_obs # (Hz)
;

In [ ]:
# Observed minus computed (O-C) residuals: TaylorIntegration.jl

# absolute
#residual_td = tdelay_jpl_obs - (tdelay_v1[delay_index]-tdelay_v1()[delay_index]+tdelay_JPL_v1[delay_index])  # (usec) <- 1.0000003 magic factor
#residual_ds = dshift_jpl_obs - (dshift_v1[doppler_index]-dshift_v1()[doppler_index]+dshift_JPL_v1[doppler_index]) # (Hz)
residual_td = tdelay_jpl_obs - (tdelay_v1[delay_index])  # (usec)
residual_ds = dshift_jpl_obs - (dshift_v1[doppler_index]) # (Hz)

# relative
rel_res_td = residual_td ./ tdelay_jpl_obs # (usec)
rel_res_ds = residual_ds ./ dshift_jpl_obs # (Hz)
;

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_v1()[delay_index],
    label="predicted (JPL DE430+s199)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_JPL_v1[delay_index],
    label="predicted (JPL DE430+s199)",
    marker=:star4
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_jpl_obs,
    label="observed (JPL)",
    legend=:topleft,
    marker=:xcross,
    yerror=dshift_jpl_obs_sigma
)
title!("Time delay vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay [microseconds]")
#xlims!(1540,1650)
#ylims!(9e7,1.1e8)

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_v1()[doppler_index],
    label="predicted (JPL DE430+s199)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_JPL_v1[doppler_index],
    label="predicted (JPL DE430+s199)",
    marker=:star4
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_jpl_obs,
    label="observed (JPL)",
    legend=:topright,
    marker=:xcross,
    yerror=dshift_jpl_obs_sigma
)
title!("Doppler shift vs time")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift [Hz]")
#xlims!(1540,1650)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    #299792.458*1e-6residual_JPL_td, yerror = 299792.458*1e-6tdelay_jpl_obs_sigma,
    residual_td(), yerror = tdelay_jpl_obs_sigma,
    marker=:xcross,
    label = "TaylorIntegration",
    legend = :topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    #299792.458*1e-6residual_JPL_td, yerror = 299792.458*1e-6tdelay_jpl_obs_sigma,
    residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "time delay (O-C) (JPL DE430+s199)",
    legend = :topright
)
title!("Time delay O-C residuals")
xlabel!("t-t0 [Julian days]")
#ylabel!("Total time delay resid. (O-C) [km]")
ylabel!("Total time delay resid. (O-C) [us]")
#xlims!(1545,1585)

In [ ]:
scatter(
    #tv_jpl_utc_jul[delay_index].-t0, 299792.458*1e-6residual_JPL_td, yerror = 299792.458*1e-6tdelay_jpl_obs_sigma,
    #tv_jpl_utc_jul[delay_index].-t0,residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    tv_jpl_utc_jul[delay_index][1:end-3].-t0, residual_JPL_td[1:end-3], yerror = tdelay_jpl_obs_sigma[1:end-3],
    marker=:cross,
    label = "time delay (O-C) (JPL DE430+s199)",
    legend = :topleft
)
#ylims!(-0,80)
#yticks!(-10:5:80)
ylabel!("Total time delay resid. (O-C) [us]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    #tv_jpl_utc_jul[delay_index].-t0, 299792.458*1e-6residual_JPL_td, yerror = 299792.458*1e-6tdelay_jpl_obs_sigma,
    tv_jpl_utc_jul[delay_index].-t0,residual_JPL_td, yerror = tdelay_jpl_obs_sigma,
    #tv_jpl_utc_jul[delay_index][1:end-3].-t0, residual_JPL_td[1:end-3], yerror = tdelay_jpl_obs_sigma[1:end-3],
    marker=:cross,
    label = "time delay (O-C) (JPL DE430+s199)",
    legend = :topleft
)
#ylims!(-0,80)
#yticks!(-10:5:80)
ylabel!("Total time delay resid. (O-C) [us]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(), yerror = dshift_jpl_obs_sigma,
    marker=:xcross,
    label = "Doppler shift (O-C) (JPL DE430+s199)",
    legend=:topright
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "Doppler shift (O-C) (JPL DE430+s199)",
    legend=:bottomright
)
title!("Doppler shift residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
#xlims!(1545,1585)

scatter(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    (299792458*1e-9/2)*residual_JPL_td,
    yerror=(299792458*1e-9/2)*Float64.(jpl_radar[del_ind,4]),
    marker=:xcross,
    label = "time delay (O-C) (JPL DE430+s199)"
)
title!("Time delay O-C residuals")
xlabel!("t-t0 [Julian days]")
ylabel!("Total time delay resid. (O-C) [km]")

scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    residual_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4]),
    marker=:xcross,
    label = "Doppler shift (O-C) (JPL DE430+s199)"
)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_JPL_td,
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="Relative time delay residual (DE430+s199)",
    marker=:xcross,
    legend=:topright
)
#scatter(
#    tv_jpl_utc_jul[delay_index].-t0,
#    rel_res_td(),
#    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
#    label="Relative time delay O-C resid. (TaylorIntegration)",
#    marker=:cross,
#    legend=:topright
#)
title!("Time delay relative O-C resid.")
xlabel!("t-t0 [Julian days]")
#ylims!(3.0e-7, 3.1e-7)


In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_td(),
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="Relative time delay O-C resid. (TaylorIntegration)",
    marker=:cross,
    legend=:topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    rel_res_JPL_td,
    yerror = tdelay_jpl_obs_sigma./tdelay_jpl_obs,
    label="Relative time delay residual (DE430+s199)",
    marker=:xcross,
    legend=:topright
)
title!("Time delay relative O-C resid.")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_ds(),
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="Relative Doppler shift O-C resid. (TI)",
    legend=:bottomright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    rel_res_JPL_ds,
    yerror = dshift_jpl_obs_sigma./dshift_jpl_obs,
    label="Relative Doppler shift O-C resid. (JPL)",
    legend=:bottomright,
    marker=:xcross
)
title!("Doppler shift relative residuals")
xlabel!("t-t0 [Julian days]")

plot(
    tv_jpl_utc_julian_unrepeated[tv_del_ind .+ 1].-t0,
    rel_res_JPL_td,
    yerror=Float64.(jpl_radar[del_ind,4])./Float64.(jpl_radar[del_ind,3]),
    label="Relative time delay residual (DE430+s199)",
    marker=:xcross,
    legend=:topright
)

scatter(
    tv_jpl_utc_julian_unrepeated[tv_dop_ind .+ 1].-t0,
    rel_res_JPL_ds,
    yerror=Float64.(jpl_radar[dop_ind,4])./Float64.(jpl_radar[dop_ind,3]),
    label="Relative Doppler shift O-C resid. (JPL)",
    legend=:bottomright,
    marker=:xcross
)

#check that HORIZONS (s#199) and downloaded SPK file for Apophis are the same
#according to this check, they differ at the end of the evaluated times by ~ 2m in position
tv_spkvhor = range(2454733.5, length=1001, stop=2456367.5)
apophis_spk_arr = Array{Float64}(undef, length(tv_spkvhor), 6)
for i in eachindex(tv_spkvhor)
    apophis_spk_arr[i,:] .= Apophis.apophis_pv(tv_spkvhor[i])
end
apophis_horizons_arr = readdlm("../wld103954.15")
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,1:3])
@show norm((apophis_spk_arr .- apophis_horizons_arr)[end,4:6])
;

# Yarkovsky $A_2$ coefficient estimation

In [ ]:
tdelay_v1[1] # τ(A2) = p(A2)

In [ ]:
tdelay_v1[1]-constant_term(tdelay_v1[1]) # δτ(A2) = τ(A2) - τ(A2=0) = p(A2) - τ(A2=0) = p[1]*A2+p[2]*A2^2+... ( p[0]=τ(A2=0) )

In [ ]:
inverse( tdelay_v1[1]-constant_term(tdelay_v1[1]) ) # A2(δτ) = q(δτ) = q[1]*δτ+q[2]*δτ^2+... (q[0]=0)

In [ ]:
td_us = tdelay_v1[delay_index]
ds_Hz = dshift_v1[doppler_index];

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δr) polynomials at each delay observation;
A2_δf_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δvr) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δf_v, residual_ds()); # A2(δf_Doppler) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    A2_del_v,
    leg=false, marker=:cross
)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(dt) x 10^14 [au/d^2]")
#ylims!(1.1minimum(A2_del_v),0)

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    A2_dop_v,
    leg=false, marker=:cross
)
xlabel!("t-t0 [Julian days]")
ylabel!("A2(df) x 10^14 [au/d^2]")
#ylims!(1.1minimum(A2_dop_v),0)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    A2_del_v,
    leg=false, marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    A2_dop_v,
    leg=false, marker=:cross
)
xlabel!("t-t0 [Julian days]")
ylabel!("A2 x 10^14 [au/d^2]")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index[1:end-3]].-t0,
    A2_del_v[1:end-3],
    leg=false, marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index[1:end-7]].-t0,
    A2_dop_v[1:end-7],
    leg=false, marker=:cross
)
xlabel!("t-t0 [Julian days]")
ylabel!("A2 x 10^14 [au/d^2]")

In [ ]:
A2_all = vcat(A2_del_v[1:end-3], A2_dop_v[1:end-7])

In [ ]:
mean(A2_all), std(A2_all)

# Intervals

In [ ]:
using IntervalArithmetic

In [ ]:
residual_td_interval = interval.(
    (tdelay_jpl_obs-tdelay_jpl_obs_sigma)-tdelay_v1[delay_index](), 
    (tdelay_jpl_obs+tdelay_jpl_obs_sigma)-tdelay_v1[delay_index]()
)

In [ ]:
residual_ds_interval = interval.(
    (dshift_jpl_obs-dshift_jpl_obs_sigma)-dshift_v1[doppler_index](),
    (dshift_jpl_obs+dshift_jpl_obs_sigma)-dshift_v1[doppler_index]()
)

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δτ_v, residual_td_interval)

In [ ]:
A2_dop_v_interval = map((x,y)->x(y), A2_δf_v, residual_ds_interval)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    mid.(A2_del_v_interval),
    yerror=radius.(A2_del_v_interval),
    label="delay",
    leg=:bottomright,
    marker=:cross
)
#ylims!(minimum(A2_del_v)-1,0)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index].-t0,
    mid.(A2_dop_v_interval),
    yerror=radius.(A2_dop_v_interval),
    label="doppler",
    marker=:cross
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlabel!("t-t0 [Julian days]")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    mid.(A2_del_v_interval),
    yerror=radius.(A2_del_v_interval),
    marker=:cross,
    leg=:bottomleft,
    label="delay"
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    mid.(A2_dop_v_interval),
    yerror=radius.(A2_dop_v_interval),
    marker=:cross,
    label="doppler"
)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-t0,
    mid.(A2_del_v_interval)[1:end-3],
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:cross,
    leg=:bottomright,
    label="delay"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-t0,
    mid.(A2_dop_v_interval)[1:end-7],
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:xcross,
    label="doppler"
)
plot!([1540, 1580], [-6.657, -6.657], label="mean adjusted value")
xlims!(1540, 1580)

In [ ]:
mean(mid.(vcat(A2_del_v_interval[1:end-3], A2_dop_v_interval[1:end-7])))

In [ ]:
mean(radius.(vcat(A2_del_v_interval[1:end-3], A2_dop_v_interval[1:end-7])))